In [1]:
!pip install rake-nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=4a46e470e12cee2873795b718982c31614cf27d9f264f47a715478c659e180e0
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [2]:
import pandas as pd
import numpy as np
from rake_nltk import Rake

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Dataset contains top 250 top rated movies from IMDB
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

In [4]:
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [5]:
# Only keep important factors for recommending movies, such as
# Title, Genre, Director, Actors and Plot
df = df[['Title','Genre','Director','Actors','Plot']]

In [6]:
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [7]:
# A library called Rake, that extracts keywords from a text
# Hence, I am going to extract keywords from the Plot column; so that I 
# only consider the most relevant keywords in the plot description

df['Plot Key Words'] = ""

for index, row in df.iterrows():
  plot = row['Plot']

  rake = Rake()

  rake.extract_keywords_from_text(plot)

  keywords_dict_score = rake.get_word_degrees()

  row["Plot Key Words"] = list(keywords_dict_score.keys())

df.drop(columns=["Plot"], inplace=True)

In [9]:
df.head(20)

,Title,Genre,Director,Actors,Plot Key Words
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","[years, two, imprisoned, men, bond, finding, s..."
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[aging, patriarch, organized, crime, dynasty, ..."
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...","[career, vito, corleone, 1920s, new, york, ear..."
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","[mysterious, past, physical, tests, menace, kn..."
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....","[justice, forcing, evidence, reconsider, misca..."
5,Schindler's List,"Biography, Drama, History",Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...","[jewish, workforce, witnessing, occupied, pola..."
6,The Lord of the Rings: The Return of the King,"Adventure, Drama, Fantasy",Peter Jackson,"Noel Appleby, Ali Astin, Sean Astin, David Aston","[men, one, ring, army, sam, draw, world, ganda..."
7,Pulp Fiction,"Crime, Drama",Quentin Tarantino,"Tim Roth, Amanda Plummer, Laura Lovelace, John...","[four, tales, lives, redemption, boxer, two, m..."
8,Fight Club,Drama,David Fincher,"Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier","[crosses, paths, insomniac, office, worker, wa..."
9,The Lord of the Rings: The Fellowship of the Ring,"Adventure, Drama, Fantasy",Peter Jackson,"Alan Howard, Noel Appleby, Sean Astin, Sala Baker","[shire, journey, save, middle, earth, destroy,..."


In [10]:
for index, row in df.iterrows():
  # putting the genres in a list
  row["Genre"] = list(row["Genre"].lower().split(','))

  # Name of Director(s) to lowercase, and remove whitespace, and put into a list
  row['Director'] = row["Director"].lower().replace(' ','').split(',')

  # Name of Actors(s) to lowercase, and remove whitespace, and put into a list
  row["Actors"] = list(row["Actors"].lower().replace(' ','').split(','))

In [11]:
df.head()

,Title,Genre,Director,Actors,Plot Key Words
0,The Shawshank Redemption,"[crime, drama]",[frankdarabont],"[timrobbins, morganfreeman, bobgunton, william...","[years, two, imprisoned, men, bond, finding, s..."
1,The Godfather,"[crime, drama]",[francisfordcoppola],"[marlonbrando, alpacino, jamescaan, richards.c...","[aging, patriarch, organized, crime, dynasty, ..."
2,The Godfather: Part II,"[crime, drama]",[francisfordcoppola],"[alpacino, robertduvall, dianekeaton, robertde...","[career, vito, corleone, 1920s, new, york, ear..."
3,The Dark Knight,"[action, crime, drama]",[christophernolan],"[christianbale, heathledger, aaroneckhart, mic...","[mysterious, past, physical, tests, menace, kn..."
4,12 Angry Men,"[crime, drama]",[sidneylumet],"[martinbalsam, johnfiedler, leej.cobb, e.g.mar...","[justice, forcing, evidence, reconsider, misca..."


In [12]:
df['Bag of Words'] = ""
columns = df.columns

for index, row in df.iterrows():
  words = ''
  for col in columns:
    if col == 'Title':
      continue
    else:
      seperator = ' '
      words = words + seperator.join(row[col]) + ' '

  # Remove the last whitespace character
  row['Bag of Words'] = words[:-1]

df.drop(columns=["Genre", "Director", "Actors", "Plot Key Words"], inplace = True)

In [14]:
df.head(20)

,Title,Bag of Words
0,The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...
1,The Godfather,crime drama francisfordcoppola marlonbrando a...
2,The Godfather: Part II,crime drama francisfordcoppola alpacino rober...
3,The Dark Knight,action crime drama christophernolan christia...
4,12 Angry Men,crime drama sidneylumet martinbalsam johnfied...
5,Schindler's List,biography drama history stevenspielberg liam...
6,The Lord of the Rings: The Return of the King,adventure drama fantasy peterjackson noelapp...
7,Pulp Fiction,crime drama quentintarantino timroth amandapl...
8,Fight Club,drama davidfincher edwardnorton bradpitt meatl...
9,The Lord of the Rings: The Fellowship of the Ring,adventure drama fantasy peterjackson alanhow...


In [15]:
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(df['Bag of Words'])

In [16]:
# get the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [17]:
print(cosine_sim)

[[1.         0.14638501 0.1315587  ... 0.05       0.05       0.05270463]
 [0.14638501 1.         0.34236839 ... 0.048795   0.048795   0.05143445]
 [0.1315587  0.34236839 1.         ... 0.0438529  0.0438529  0.04622502]
 ...
 [0.05       0.048795   0.0438529  ... 1.         0.05       0.05270463]
 [0.05       0.048795   0.0438529  ... 0.05       1.         0.05270463]
 [0.05270463 0.05143445 0.04622502 ... 0.05270463 0.05270463 1.        ]]


In [18]:
indices = pd.Series(df["Title"])
ind = indices
ind[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [19]:
def recommendations(title):
  recommended_movies = []

  # Retrieves the index of the movie in the dataset
  idx = indices[indices == title].index[0]

  # Retrieves all the scores from the cosine similarity matrix,
  # convert to a pd.Series, and sort the scores in descending order
  score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
  
  # Retrieves the highest 10 scores, excluding its own movie
  top_10_indexes = list(score_series.iloc[1:11].index)

  for i in top_10_indexes:
    recommended_movies.append(list(df["Title"])[i])

  return recommended_movies

In [21]:
recommendations('Inception')

['Mad Max: Fury Road',
 'Interstellar',
 'The Dark Knight Rises',
 'Guardians of the Galaxy',
 'Aliens',
 'Guardians of the Galaxy Vol. 2',
 'Spider-Man: Homecoming',
 'The Prestige',
 'The Matrix',
 'The Revenant']